### Después de este bloque de código empiezan los cambios

In [ ]:
dir.create("modelitos_prod", showWarnings = FALSE)

param_completo <- copy(PARAM$prod$lgbm$param_completo)  # ✓ Corregido (era bprod)

cat("Iniciando entrenamiento de", length(PARAM$prod$semillas), "modelos...\n")

for(i in seq_along(PARAM$prod$semillas)) {
  sem <- PARAM$prod$semillas[i]
  arch_modelo <- paste0("./modelitos_prod/mod_", sem, ".txt")
  
  if(!file.exists(arch_modelo)) {
    cat("Entrenando modelo", i, "de", length(PARAM$prod$semillas), "\r")
    
    param_completo$seed <- sem
    
    modelito <- lgb.train(
      data = dtrain_prod,
      param = param_completo
    )
    
    lgb.save(modelito, filename = arch_modelo)
    rm(modelito)
    gc()
  }
}

cat("\nModelos entrenados. Verificando...\n")
archivos <- list.files("./modelitos_prod/", pattern = "mod_.*\\.txt")
cat("Modelos encontrados:", length(archivos), "\n")

Sys.time()

### A partir de acá van los nuevos bloques

In [ ]:
# Preparar datos de future (una sola vez)
dfuture_prod <- dataset[foto_mes %in% PARAM$prod$future]
cat("Clientes en producción:", nrow(dfuture_prod), "\n")

# Agregar canaritos si es necesario
if(PARAM$prod$lgbm$qcanaritos > 0) {
  filas <- nrow(dfuture_prod)
  for(i in seq(PARAM$prod$lgbm$qcanaritos)) {
    dfuture_prod[, paste0("canarito_", i) := runif(filas)]
  }
}

# Agregar ganancia para calcular meseta
dfuture_prod[, gan := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]

# Preparar matriz de features
mfuture_prod <- data.matrix(dfuture_prod[, campos_buenos, with = FALSE])

# Vector para guardar los 10 máximos de meseta
vector_ganancias_meseta <- numeric(length(PARAM$prod$semillas))

cat("\nCalculando ganancias con meseta suavizada...\n")

#### Entrena los 10 modelos (1 por cada semilla)

In [ ]:
# Loop: predicción individual por cada modelo
for(i in seq_along(PARAM$prod$semillas)) {
  sem <- PARAM$prod$semillas[i]
  arch_modelo <- paste0("./modelitos_prod/mod_", sem, ".txt")
  
  cat("Procesando modelo", i, "de", length(PARAM$prod$semillas), "\r")
  
  # Cargar modelo individual
  modelo <- lgb.load(arch_modelo)
  
  # Predecir (SIN promediar)
  vpred <- predict(modelo, mfuture_prod)
  
  # Crear tabla de ganancia
  tb <- data.table(gan = dfuture_prod$gan, prob = vpred)
  setorder(tb, -prob)
  tb[, gan_acum := cumsum(gan)]
  
  # Calcular meseta suavizada (moving average de 2001)
  tb[, gan_meseta := frollmean(
    x = gan_acum,
    n = 2001,
    align = "center",
    na.rm = TRUE
  )]
  
  # Guardar máximo de meseta
  vector_ganancias_meseta[i] <- max(tb$gan_meseta, na.rm = TRUE)
  
  rm(modelo, vpred, tb)
  gc()
}

cat("\n✅ Ganancias de meseta calculadas\n")

#### Muestro resultados

In [ ]:
# Mostrar resultados
cat("\n")
cat(strrep("=", 60), "\n")  # ✅ Correcto en R
cat("RESULTADOS - VECTOR DE GANANCIAS\n")
cat(strrep("=", 60), "\n")

cat("\nGanancias de meseta por modelo:\n")
for(i in seq_along(vector_ganancias_meseta)) {
  cat(sprintf("Modelo %2d (semilla %d): $%s\n", 
              i, 
              PARAM$prod$semillas[i],
              format(vector_ganancias_meseta[i], big.mark=",", scientific=FALSE)))
}

cat("\nEstadísticas:\n")
cat("  Media:   $", format(mean(vector_ganancias_meseta), big.mark=",", scientific=FALSE), "\n")
cat("  Mediana: $", format(median(vector_ganancias_meseta), big.mark=",", scientific=FALSE), "\n")
cat("  SD:      $", format(sd(vector_ganancias_meseta), big.mark=",", scientific=FALSE), "\n")
cat("  Mín:     $", format(min(vector_ganancias_meseta), big.mark=",", scientific=FALSE), "\n")
cat("  Máx:     $", format(max(vector_ganancias_meseta), big.mark=",", scientific=FALSE), "\n")

# Guardar vector para comparaciones futuras
fwrite(
  data.table(
    modelo = 1:length(vector_ganancias_meseta),
    semilla = PARAM$prod$semillas,
    ganancia_meseta = vector_ganancias_meseta
  ),
  file = "ganancias_meseta_prod.txt",
  sep = "\t"
)

cat("\n✅ Resultados guardados en: ganancias_meseta_prod.txt\n")

#### Curvas de ganancia y sus mesetas

In [ ]:
library(data.table)
library(ggplot2)

# Preparar data.table para acumular todas las curvas
tb_todas <- data.table()

cat("Calculando curvas para", length(PARAM$prod$semillas), "modelos...\n")

for(i in seq_along(PARAM$prod$semillas)) {
  sem <- PARAM$prod$semillas[i]
  
  cat("Modelo", i, "de", length(PARAM$prod$semillas), "\r")
  
  # Cargar modelo y predecir
  modelo <- lgb.load(paste0("modelitos_prod/mod_", sem, ".txt"))
  vpred <- predict(modelo, mfuture_prod)
  
  # Crear tabla
  tb <- data.table(gan = dfuture_prod$gan, prob = vpred)
  setorder(tb, -prob)
  tb[, gan_acum := cumsum(gan)]
  
  # Meseta suavizada
  tb[, gan_meseta := frollmean(
    x = gan_acum,
    n = 2001,
    align = "center",
    na.rm = TRUE
  )]
  
  # Agregar identificador de modelo
  tb[, `:=`(
    envio = .I,
    modelo = i,
    semilla = sem
  )]
  
  # Acumular
  tb_todas <- rbind(tb_todas, tb[, .(envio, gan_acum, gan_meseta, modelo, semilla)])
  
  rm(modelo, vpred, tb)
  gc()
}

cat("\n✅ Curvas calculadas\n")

# Gráfico con todas las curvas
ggplot(tb_todas) +
  geom_line(aes(envio, gan_acum, group = modelo, color = as.factor(modelo)), 
            alpha = 0.3) +
  geom_line(aes(envio, gan_meseta, group = modelo, color = as.factor(modelo)), 
            linewidth = 1) +
  coord_cartesian(xlim = c(6000, 19000), ylim = c(325000000, 420000000)) +
  labs(
    title = paste("Curvas de ganancia -", length(PARAM$prod$semillas), "modelos"),
    subtitle = "Líneas finas: gan_acum | Líneas gruesas: meseta suavizada",
    x = "Envíos",
    y = "Ganancia acumulada",
    color = "Modelo"
  ) +
  theme_minimal() +
  theme(legend.position = "right")